In [1]:
import torch
from torch import nn

In [3]:
# 二维卷积：H 和 W 两个方向做卷积
conv2d = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=2, padding=1)

In [5]:
# 模拟图像 [N, C, H, W]
X = torch.randn(1, 3, 5, 5)

In [7]:
conv2d(X).shape

torch.Size([1, 2, 3, 3])

In [9]:
conv2d.weight.shape

torch.Size([2, 3, 3, 3])

In [11]:
conv2d.bias.shape

torch.Size([2])

In [14]:
# 批规范化层
bn = nn.BatchNorm2d(num_features=2)

In [15]:
bn.weight

Parameter containing:
tensor([1., 1.], requires_grad=True)

In [16]:
bn.bias

Parameter containing:
tensor([0., 0.], requires_grad=True)

In [17]:
torch.relu

<function torch._VariableFunctionsClass.relu>

In [20]:
# 最大池化、亚采样、把图像变小，丢掉不重要的特征，保留最重要的特征
mp = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

In [22]:
X = torch.randint(low=0, high=101, size=(1, 1, 4, 4),dtype=torch.float32)

In [24]:
# [N, C, H, W]
X.shape

torch.Size([1, 1, 4, 4])

In [26]:
X

tensor([[[[91.,  2.,  5., 29.],
          [ 6., 99., 52.,  9.],
          [61., 75., 51., 94.],
          [72., 38., 26., 31.]]]])

In [27]:
mp(X)

tensor([[[[99., 52.],
          [75., 94.]]]])

In [28]:
avg_pool = nn.AvgPool2d(kernel_size=2, stride=2, padding=0)

In [29]:
X

tensor([[[[91.,  2.,  5., 29.],
          [ 6., 99., 52.,  9.],
          [61., 75., 51., 94.],
          [72., 38., 26., 31.]]]])

In [30]:
avg_pool(X)

tensor([[[[49.5000, 23.7500],
          [61.5000, 50.5000]]]])

In [32]:
X[:, :, :2, :2].mean()

tensor(49.5000)

### 搭建一个LeNet5

![LeNet5](lenet5.png)

In [33]:
import torch
from torch import nn

In [75]:
class Model(nn.Module):
    """
        自定义一个神经网络
    """
    def __init__(self, in_channels=1, n_classes=10):
        """
            初始化
        """
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                               out_channels=6, 
                               kernel_size=5,
                               stride=1,
                               padding=0)
        self.mp1 = nn.MaxPool2d(kernel_size=2, 
                                stride=2,
                                padding=0)
        self.conv2 = nn.Conv2d(in_channels=6,
                              out_channels=16,
                              kernel_size=5,
                              stride=1,
                              padding=0)
        self.mp2 = nn.MaxPool2d(kernel_size=2, 
                               stride=2,
                               padding=0)
        self.flatten = nn.Flatten(start_dim=1,
                                  end_dim=-1)
        self.fc1 = nn.Linear(in_features=400,
                            out_features=120)
        self.fc2 = nn.Linear(in_features=120,
                            out_features=84)
        self.fc3 = nn.Linear(in_features=84,
                            out_features=n_classes)
    def forward(self, x):
        """
            前向传播
        """
        x = self.conv1(x)
        x = self.mp1(x)
        x = self.conv2(x)
        x = self.mp2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [81]:
class Model(nn.Module):
    """
        自定义一个神经网络
    """
    def __init__(self, in_channels=1, n_classes=10):
        """
            初始化
        """
        super(Model, self).__init__()

        # 1. 特征抽取
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, 
                               out_channels=6, 
                               kernel_size=5,
                               stride=1,
                               padding=0),
            nn.MaxPool2d(kernel_size=2, 
                                stride=2,
                                padding=0),
            nn.Conv2d(in_channels=6,
                              out_channels=16,
                              kernel_size=5,
                              stride=1,
                              padding=0),
            nn.MaxPool2d(kernel_size=2, 
                               stride=2,
                               padding=0)
        )

        # 2. 分类输出
        self.classifier = nn.Sequential(
            nn.Flatten(start_dim=1, end_dim=-1),
            nn.Linear(in_features=400, out_features=120),
            nn.Linear(in_features=120, out_features=84),
            nn.Linear(in_features=84, out_features=n_classes)
        )

    def forward(self, x):
        """
            前向传播
        """
        # 1. 先做特征抽取
        x = self.feature_extractor(x)
        # 2. 再做分类回归
        x = self.classifier(x)
        return x

In [82]:
model = Model(in_channels=1)

In [83]:
X = torch.randn(2, 1, 32, 32)

In [84]:
y_pred = model(X)

In [85]:
y_pred.shape

torch.Size([2, 10])

In [86]:
model

Model(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Linear(in_features=84, out_features=10, bias=True)
  )
)